# Camera Image Processing

**Learning Objective**: Understand basic computer vision operations for robotics

**Estimated Time**: 10 minutes

This notebook demonstrates basic image processing that robots perform 30-60 times per second for vision-based control.

In [ ]:
# Install dependencies
!pip install numpy==1.24.0 matplotlib==3.7.0 pillow==9.5.0

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageFilter

In [ ]:
def create_synthetic_scene() -> np.ndarray:
    """
    Create a synthetic robot vision scene.
    """
    width, height = 640, 480
    image = np.ones((height, width, 3), dtype=np.uint8) * 200
    
    # Add table
    image[300:450, 100:540] = [139, 90, 43]
    
    # Add objects
    image[250:300, 150:200] = [220, 50, 50]  # Red cube
    image[260:310, 400:450] = [50, 50, 220]  # Blue cylinder
    
    # Green sphere (center)
    center_y, center_x = 280, 320
    radius = 30
    y, x = np.ogrid[:height, :width]
    mask = (x - center_x)**2 + (y - center_y)**2 <= radius**2
    image[mask] = [50, 220, 50]
    
    return image

def rgb_to_grayscale(image: np.ndarray) -> np.ndarray:
    """
    Convert RGB to grayscale using luminosity method.
    """
    return (0.299 * image[:, :, 0] + 
            0.587 * image[:, :, 1] + 
            0.114 * image[:, :, 2]).astype(np.uint8)

def simple_edge_detection(image: np.ndarray, threshold: int = 30) -> np.ndarray:
    """
    Detect edges using gradient-based method.
    """
    pil_img = Image.fromarray(image)
    edges_pil = pil_img.filter(ImageFilter.FIND_EDGES)
    edges = np.array(edges_pil)
    return (edges > threshold).astype(np.uint8) * 255

def apply_blur(image: np.ndarray, radius: int = 2) -> np.ndarray:
    """
    Apply Gaussian blur to reduce noise.
    """
    pil_img = Image.fromarray(image)
    blurred_pil = pil_img.filter(ImageFilter.GaussianBlur(radius=radius))
    return np.array(blurred_pil)

def calculate_processing_stats(edges: np.ndarray) -> dict:
    """
    Calculate statistics from processed image.
    """
    edge_pixels = np.sum(edges > 0)
    total_pixels = edges.size
    return {
        'edge_pixels': edge_pixels,
        'edge_percentage': 100.0 * edge_pixels / total_pixels,
        'total_pixels': total_pixels
    }

In [ ]:
# Create and process scene
original_image = create_synthetic_scene()
grayscale = rgb_to_grayscale(original_image)
grayscale_blurred = apply_blur(grayscale, radius=1)
edges = simple_edge_detection(grayscale_blurred, threshold=20)

# Calculate statistics
stats = calculate_processing_stats(edges)

print("=" * 60)
print("CAMERA IMAGE PROCESSING: Robot Vision Basics")
print("=" * 60)
print("\nProcessing steps:")
print("  1. Capture RGB image (640x480)")
print("  2. Convert to grayscale (reduce data)")
print("  3. Apply edge detection (find objects)")
print(f"\nProcessing Results:")
print(f"  • Image Resolution: {original_image.shape[1]}x{original_image.shape[0]}")
print(f"  • Total Pixels: {stats['total_pixels']:,}")
print(f"  • Edge Pixels Detected: {stats['edge_pixels']:,}")
print(f"  • Edge Density: {stats['edge_percentage']:.2f}%")
print("\n" + "=" * 60)
print("KEY INSIGHT: Robots process images at 30-60 FPS")
print("Must be fast and efficient for real-time control!")
print("=" * 60)

In [ ]:
# Visualize processing pipeline
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].imshow(original_image)
axes[0].set_title('Original Camera Image\n(RGB from Intel RealSense)', fontsize=12, fontweight='bold')
axes[0].axis('off')

axes[1].imshow(grayscale, cmap='gray')
axes[1].set_title('Grayscale Conversion\n(First preprocessing step)', fontsize=12, fontweight='bold')
axes[1].axis('off')

axes[2].imshow(edges, cmap='gray')
axes[2].set_title('Edge Detection\n(Object boundaries)', fontsize=12, fontweight='bold')
axes[2].axis('off')

plt.suptitle('Camera Image Processing Pipeline for Robot Vision', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

## Expected Output

You should see:
1. **Statistics**: 307,200 total pixels, ~5-10% edge pixels
2. **Visualization**: Original RGB → Grayscale → Edge map

**Key Takeaway**: Robots process camera images continuously for object detection, navigation, and manipulation. Speed and efficiency are critical for real-time control.